In [1]:
import os

import numpy as np
import xarray as xr
from pyrte_rrtmgp.rrtmgp_gas_optics import GasOpticsFiles, load_gas_optics
from pyrte_rrtmgp.rrtmgp_data import download_rrtmgp_data
from pyrte_rrtmgp.rte_solver import RTESolver
from pyrte_rrtmgp.all_skys_funcs import (
    compute_clouds,
    compute_cloud_optics,
    combine_optical_props,
)
from pyrte_rrtmgp.utils import compute_profiles, create_gas_dataset

rte_rrtmgp_dir = download_rrtmgp_data()

ncol = 24
nlay = 72
# Compute profiles
profiles = compute_profiles(300, ncol, nlay)

lw_no_aerosol = (
    f"{rte_rrtmgp_dir}/examples/all-sky/reference/rrtmgp-allsky-lw-no-aerosols.nc"
)
ref_data = xr.load_dataset(lw_no_aerosol, decode_cf=False)

gas_optics_lw = load_gas_optics(gas_optics_file=GasOpticsFiles.LW_G256)
lw_clouds = os.path.join(rte_rrtmgp_dir, "rrtmgp-clouds-lw-bnd.nc")

# Create dataset with gas concentrations
gas_values = {
    "co2": 348e-6,
    "ch4": 1650e-9,
    "n2o": 306e-9,
    "n2": 0.7808,
    "o2": 0.2095,
    "co": 0.0,
}

gases = create_gas_dataset(gas_values, dims={"site": ncol, "layer": nlay})

atmosphere = xr.merge([profiles, gases])
top_at_1 = (
    atmosphere["pres_layer"].values[0, 0] < atmosphere["pres_layer"].values[0, -1]
)

t_sfc = profiles["temp_level"][:, nlay if top_at_1 else 0]
atmosphere["surface_temperature"] = xr.DataArray(t_sfc, dims=["site"])

gas_optics_lw = load_gas_optics(gas_optics_file=GasOpticsFiles.LW_G256)
clear_sky_optical_props = gas_optics_lw.gas_optics.compute(
    atmosphere, problem_type="absorption", add_to_input=False
)

cloud_optics = xr.load_dataset(lw_clouds)

t_sfc = profiles["temp_level"][0, nlay if top_at_1 else 0]

emis_sfc = xr.DataArray(
    0.98,
    dims=["site", "level"],
    coords={"site": np.arange(ncol), "level": np.arange(nlay + 1)},
    name="surface_albedo_dir",
)


cloud_properties = compute_clouds(
    cloud_optics, ncol, nlay, profiles["pres_layer"], profiles["temp_layer"]
)

tau = compute_cloud_optics(cloud_properties, cloud_optics)

# Create dataset with optical properties
clouds_optical_props = xr.Dataset(
    {
        "tau": (["site", "layer", "gpt"], tau),
    }
)

clear_sky_optical_props["surface_emissivity"] = 0.98

# Increment the optical properties
combine_optical_props(clouds_optical_props, clear_sky_optical_props)

solver = RTESolver()
fluxes = solver.solve(clear_sky_optical_props, add_to_input=False)


lw_no_aerosol = (
    f"{rte_rrtmgp_dir}/examples/all-sky/reference/rrtmgp-allsky-lw-no-aerosols.nc"
)
ref_data = xr.load_dataset(lw_no_aerosol, decode_cf=False)

# Assertions
assert np.isclose(
    fluxes["lw_flux_up"].values, ref_data["lw_flux_up"].values.T, atol=1e-7
).all()
assert np.isclose(
    fluxes["lw_flux_down"].values, ref_data["lw_flux_dn"].values.T, atol=1e-7
).all()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/josue/Documents/makepath/rte-python/pyRTE-RRTMGP/examples/clear_sky.nc'

In [1]:
import os

import numpy as np
import xarray as xr
from pyrte_rrtmgp.rrtmgp_gas_optics import GasOpticsFiles, load_gas_optics
from pyrte_rrtmgp.rrtmgp_data import download_rrtmgp_data
from pyrte_rrtmgp.rte_solver import RTESolver
from pyrte_rrtmgp.all_skys_funcs import (
    compute_clouds,
    compute_cloud_optics,
    combine_optical_props,
    delta_scale_optical_props,
)
from pyrte_rrtmgp.utils import compute_profiles, create_gas_dataset

rte_rrtmgp_dir = download_rrtmgp_data()

ncol = 24
nlay = 72

# Compute profiles
profiles = compute_profiles(300, ncol, nlay)

sw_no_aerosol = (
    f"{rte_rrtmgp_dir}/examples/all-sky/reference/rrtmgp-allsky-sw-no-aerosols.nc"
)
ref_data = xr.load_dataset(sw_no_aerosol, decode_cf=False)

gas_optics_sw = load_gas_optics(gas_optics_file=GasOpticsFiles.SW_G224)
sw_clouds = os.path.join(rte_rrtmgp_dir, "rrtmgp-clouds-sw-bnd.nc")
ngpt = gas_optics_sw.sizes["gpt"]

# Create dataset with gas concentrations
gas_values = {
    "co2": 348e-6,
    "ch4": 1650e-9,
    "n2o": 306e-9,
    "n2": 0.7808,
    "o2": 0.2095,
    "co": 0.0,
}

gases = create_gas_dataset(gas_values, dims={"site": ncol, "layer": nlay})

atmosphere = xr.merge([profiles, gases])

# For SW calculations
sfc_alb_dir = xr.DataArray(
    0.06,
    dims=["site", "gpt"],
    coords={"site": np.arange(ncol), "gpt": np.arange(ngpt)},
    name="surface_albedo_dir",
)

sfc_alb_dif = xr.DataArray(
    0.06,
    dims=["site", "gpt"],
    coords={"site": np.arange(ncol), "gpt": np.arange(ngpt)},
    name="surface_albedo_dif",
)

mu0 = xr.DataArray(
    0.86,
    dims=["site", "layer"],
    coords={"site": np.arange(ncol), "layer": np.arange(nlay)},
    name="mu0",
)

atmosphere["mu0"] = mu0
atmosphere["surface_albedo_dir"] = sfc_alb_dir
atmosphere["surface_albedo_dif"] = sfc_alb_dif

top_at_1 = (
    atmosphere["pres_layer"].values[0, 0] < atmosphere["pres_layer"].values[0, -1]
)

clear_sky_optical_props = gas_optics_sw.gas_optics.compute(
    atmosphere, problem_type="two-stream", add_to_input=False
)

cloud_optics = xr.load_dataset(sw_clouds)

cloud_properties = compute_clouds(
    cloud_optics, ncol, nlay, profiles["pres_layer"], profiles["temp_layer"]
)

tau, ssa, g = compute_cloud_optics(cloud_properties, cloud_optics, lw=False)

# Create dataset with optical properties
clouds_optical_props = xr.Dataset(
    {
        "ssa": (["site", "layer", "gpt"], ssa),
        "g": (["site", "layer", "gpt"], g),
        "tau": (["site", "layer", "gpt"], tau),
    }
)

# Increment the optical properties
delta_scale_optical_props(clouds_optical_props)
combine_optical_props(clouds_optical_props, clear_sky_optical_props)

solver = RTESolver()
fluxes = solver.solve(clear_sky_optical_props, add_to_input=False)

# Assertions
assert np.isclose(
    fluxes["sw_flux_up"].values, ref_data["sw_flux_up"].values.T, atol=1e-7
).all()
assert np.isclose(
    fluxes["sw_flux_down"].values, ref_data["sw_flux_dn"].values.T, atol=1e-7
).all()